In [11]:
import sqlite3 as sq

In [12]:
conn = sq.connect('space_tourism.db')
cursor = conn.cursor()

In [13]:

try:
    # Create tables in proper order to handle foreign key dependencies
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "statuses" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL UNIQUE,
        PRIMARY KEY("id")
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "positions" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL UNIQUE,
        "salary" INTEGER NOT NULL,
        PRIMARY KEY("id")
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "space_objects" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL,
        PRIMARY KEY("id")
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "airports" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL,
        PRIMARY KEY("id")
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "products" (
        "id" INTEGER NOT NULL UNIQUE,
        "product" VARCHAR NOT NULL,
        "price" INTEGER NOT NULL,
        PRIMARY KEY("id")
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "reasons" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL UNIQUE,
        "cost" INTEGER NOT NULL,
        "product_id" INTEGER NOT NULL,
        "count" INTEGER NOT NULL,
        PRIMARY KEY("id"),
        FOREIGN KEY ("product_id") REFERENCES "products"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "spaceships" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL,
        "status_id" INTEGER NOT NULL,
        "human_places" INTEGER NOT NULL,
        "petrol_tank_size" INTEGER NOT NULL,
        PRIMARY KEY("id"),
        FOREIGN KEY ("status_id") REFERENCES "statuses"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "clients" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL,
        "surname" VARCHAR NOT NULL,
        PRIMARY KEY("id")
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "tours" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL,
        "obj_id" INTEGER NOT NULL,
        "airport_id" INTEGER NOT NULL,
        "spaceship_id" INTEGER NOT NULL,
        "cost" INTEGER NOT NULL,
        "date" DATE NOT NULL,
        PRIMARY KEY("id"),
        FOREIGN KEY ("airport_id") REFERENCES "airports"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION,
        FOREIGN KEY ("spaceship_id") REFERENCES "spaceships"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION,
        FOREIGN KEY ("obj_id") REFERENCES "space_objects"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "employees" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL,
        "surname" VARCHAR NOT NULL,
        "tour_id" INTEGER,
        "position_id" INTEGER NOT NULL,
        PRIMARY KEY("id"),
        FOREIGN KEY ("tour_id") REFERENCES "tours"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION,
        FOREIGN KEY ("position_id") REFERENCES "positions"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "maintain_employees" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL,
        "surname" VARCHAR NOT NULL,
        "ticket_id" INTEGER DEFAULT NULL,
        "position_id" INTEGER NOT NULL,
        PRIMARY KEY("id"),
        FOREIGN KEY ("position_id") REFERENCES "positions"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "maintain_tickets" (
        "id" INTEGER NOT NULL UNIQUE,
        "spaceship_id" INTEGER NOT NULL,
        "reason_id" INTEGER NOT NULL,
        PRIMARY KEY("id"),
        FOREIGN KEY ("reason_id") REFERENCES "reasons"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION,
        FOREIGN KEY ("spaceship_id") REFERENCES "spaceships"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION,
        FOREIGN KEY ("id") REFERENCES "maintain_employees"("ticket_id")
        ON UPDATE NO ACTION ON DELETE NO ACTION
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "tour_tickets" (
        "tour_id" INTEGER NOT NULL,
        "price" INTEGER NOT NULL,
        "client_id" INTEGER NOT NULL,
        FOREIGN KEY ("client_id") REFERENCES "clients"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION,
        FOREIGN KEY ("tour_id") REFERENCES "tours"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "reviews" (
        "text" VARCHAR NOT NULL,
        "client_id" INTEGER NOT NULL,
        "tour_id" INTEGER NOT NULL,
        "date" DATE NOT NULL,
        FOREIGN KEY ("client_id") REFERENCES "clients"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION,
        FOREIGN KEY ("tour_id") REFERENCES "tours"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS "candidats" (
        "id" INTEGER NOT NULL UNIQUE,
        "name" VARCHAR NOT NULL,
        "surname" VARCHAR NOT NULL,
        "position_id" INTEGER NOT NULL,
        "mark" INTEGER NOT NULL,
        PRIMARY KEY("id"),
        FOREIGN KEY ("position_id") REFERENCES "positions"("id")
        ON UPDATE NO ACTION ON DELETE NO ACTION
    )
    ''')

    # Commit the changes
    conn.commit()
    print("All tables created successfully in space_tourism.db!")
    
except sq.Error as e:
    print(f"An error occurred: {e}")
    
finally:
    # Close the connection
    conn.close()

All tables created successfully in space_tourism.db!


In [14]:
conn.close()

In [15]:
import sqlite3
import random
from datetime import datetime, timedelta

def generate_mock_data():
    conn = sqlite3.connect('space_tourism.db')
    cursor = conn.cursor()
    
    try:
        # Clear existing data (optional)
        cursor.execute("PRAGMA foreign_keys = OFF")
        tables = ["reviews", "tour_tickets", "maintain_employees", "maintain_tickets", 
                "employees", "tours", "clients", "spaceships", "airports", 
                "space_objects", "reasons", "products", "positions", "statuses",
                "candidats"]
        for table in tables:
            cursor.execute(f"DELETE FROM {table}")
        cursor.execute("PRAGMA foreign_keys = ON")
        
        # 1. Statuses
        statuses = [
            (1, "Ready"),
            (2, "Maintenance"),
            (3, "In Flight"),
            (4, "Retired")
        ]
        cursor.executemany("INSERT INTO statuses VALUES (?, ?)", statuses)
        
        # 2. Positions
        positions = [
            (1, "Pilot", 100000),
            (2, "Co-Pilot", 80000),
            (3, "Flight Attendant", 50000),
            (4, "Engineer", 90000),
            (5, "Mechanic", 60000),
            (6, "Tour Guide", 45000)
        ]
        cursor.executemany("INSERT INTO positions VALUES (?, ?, ?)", positions)
        
        # 3. Space Objects
        space_objects = [
            (1, "Moon"),
            (2, "Mars"),
            (3, "International Space Station"),
            (4, "Venus Orbit"),
            (5, "Jupiter Flyby")
        ]
        cursor.executemany("INSERT INTO space_objects VALUES (?, ?)", space_objects)
        
        # 4. Airports
        airports = [
            (1, "Starport Alpha"),
            (2, "Cosmodrome Beta"),
            (3, "Galactic Central"),
            (4, "Orbital One")
        ]
        cursor.executemany("INSERT INTO airports VALUES (?, ?)", airports)
        
        # 5. Products
        products = [
            (1, "Engine Parts", 5000),
            (2, "Life Support System", 12000),
            (3, "Navigation Computer", 8000),
            (4, "Hull Plating", 3000)
        ]
        cursor.executemany("INSERT INTO products VALUES (?, ?, ?)", products)
        
        # 6. Reasons
        reasons = [
            (1, "Engine Overhaul", 25000, 1, 1),
            (2, "Life Support Maintenance", 18000, 2, 1),
            (3, "Navigation System Upgrade", 15000, 3, 1),
            (4, "Hull Repair", 9000, 4, 3)
        ]
        cursor.executemany("INSERT INTO reasons VALUES (?, ?, ?, ?, ?)", reasons)
        
        # 7. Spaceships
        spaceships = [
            (1, "Star Voyager", 1, 50, 5000),
            (2, "Galactic Cruiser", 1, 75, 7500),
            (3, "Cosmic Explorer", 2, 30, 4000),
            (4, "Orbital Shuttle", 1, 20, 3000)
        ]
        cursor.executemany("INSERT INTO spaceships VALUES (?, ?, ?, ?, ?)", spaceships)
        
        # 8. Clients
        clients = []
        first_names = ["John", "Emma", "Michael", "Sophia", "William", "Olivia"]
        last_names = ["Smith", "Johnson", "Williams", "Brown", "Jones", "Garcia"]
        for i in range(1, 21):
            clients.append((
                i,
                random.choice(first_names),
                random.choice(last_names)
            ))
        cursor.executemany("INSERT INTO clients VALUES (?, ?, ?)", clients)
        
        # 9. Tours
        tours = []
        tour_names = [
            "Lunar Getaway", 
            "Mars Expedition", 
            "ISS Experience", 
            "Venus Sightseeing", 
            "Jupiter Adventure"
        ]
        start_date = datetime.now() + timedelta(days=10)
        for i in range(1, 11):
            tour_date = start_date + timedelta(days=random.randint(0, 90))
            tours.append((
                i,
                random.choice(tour_names),
                random.randint(1, 5),  # obj_id
                random.randint(1, 4),  # airport_id
                random.randint(1, 4),  # spaceship_id
                random.randint(5000, 20000),  # cost
                tour_date.strftime("%Y-%m-%d")  # date
            ))
        cursor.executemany("INSERT INTO tours VALUES (?, ?, ?, ?, ?, ?, ?)", tours)
        
        # 10. Employees
        employees = []
        emp_first_names = ["David", "Sarah", "Robert", "Jennifer", "James", "Lisa"]
        emp_last_names = ["Miller", "Davis", "Wilson", "Anderson", "Thomas", "Taylor"]
        for i in range(1, 16):
            employees.append((
                i,
                random.choice(emp_first_names),
                random.choice(emp_last_names),
                random.randint(1, 10) if random.random() > 0.3 else None,  # tour_id
                random.randint(1, 6)  # position_id
            ))
        cursor.executemany("INSERT INTO employees VALUES (?, ?, ?, ?, ?)", employees)
        
        # 11. Maintain Employees
        maint_employees = []
        for i in range(1, 6):
            maint_employees.append((
                i,
                random.choice(emp_first_names),
                random.choice(emp_last_names),
                None,  # ticket_id will be set after maintain_tickets
                random.choice([4, 5])  # position_id (engineer or mechanic)
            ))
        cursor.executemany("INSERT INTO maintain_employees VALUES (?, ?, ?, ?, ?)", maint_employees)
        
        # 12. Maintain Tickets
        maintain_tickets = []
        for i in range(1, 6):
            maintain_tickets.append((
                i,
                random.randint(1, 4),  # spaceship_id
                random.randint(1, 4)  # reason_id
            ))
        cursor.executemany("INSERT INTO maintain_tickets VALUES (?, ?, ?)", maintain_tickets)
        
        # Update maintain_employees with ticket_ids
        for i in range(1, 6):
            cursor.execute("UPDATE maintain_employees SET ticket_id = ? WHERE id = ?", (i, i))
        
        # 13. Tour Tickets
        tour_tickets = []
        for i in range(1, 31):
            tour_tickets.append((
                random.randint(1, 10),  # tour_id
                random.randint(4500, 22000),  # price
                random.randint(1, 20)  # client_id
            ))
        cursor.executemany("INSERT INTO tour_tickets VALUES (?, ?, ?)", tour_tickets)
        
        # 14. Reviews
        reviews = []
        review_texts = [
            "Amazing experience!",
            "Once in a lifetime opportunity.",
            "The food could be better.",
            "Worth every penny!",
            "The crew was fantastic.",
            "Some technical issues during flight."
        ]
        for i in range(1, 21):
            review_date = (datetime.now() - timedelta(days=random.randint(1, 30))).strftime("%Y-%m-%d")
            reviews.append((
                random.choice(review_texts),
                random.randint(1, 20),  # client_id
                random.randint(1, 10),  # tour_id
                review_date
            ))
        cursor.executemany("INSERT INTO reviews VALUES (?, ?, ?, ?)", reviews)
        
        # 15. Candidates
        candidates = []
        for i in range(1, 11):
            candidates.append((
                i,
                random.choice(emp_first_names),
                random.choice(emp_last_names),
                random.randint(1, 6),  # position_id
                random.randint(3, 10)  # mark
            ))
        cursor.executemany("INSERT INTO candidats VALUES (?, ?, ?, ?, ?)", candidates)
        
        conn.commit()
        print("Successfully generated and inserted mock data!")
        
    except sqlite3.Error as e:
        print(f"Error generating mock data: {e}")
    finally:
        conn.close()

if __name__ == "__main__":
    generate_mock_data()

Successfully generated and inserted mock data!


In [16]:
conn.close()